In [0]:
from random import randint
from random import choice
import random

#This is the Elevator class.
class Elevator:
  
    #sets the initial values of the Elevator class
    def __init__(self, max_capacity, position):
        self.position = position #the position (floor) of the elevator
        self.max_capacity = max_capacity #the max number of people that the elevator can carry at a given time
        self.load = 0 #initially the elevator has no passengers
        self.time = 0 #the elevator starts at time = 0
        self.door = 0 #closed = 0, doors open = 1, the doors start closed
    
    #allows the elevator to move up and down
    def move_one(self, direction): 
        """
        If traveling up, direction = +1.
        If traveling down, direction = -1.
        """
        self.position += direction  #changes the position of the elevator as it moves up and down
        self.time += 3  #this is the amount of time (in seconds) that it takes to move between each floor
             
    #used in the first strategy to pick up passengers
    def pick_up(self, passengers):
        for passenger in passengers: #loops through all passengers to check if anyone needs to be picked up
            if (self.position == passenger.start) & (passenger.traveling == 0) & (passenger.arrived == 0) & (self.load < self.max_capacity): #the elevator will only pick someone if the elevator is not full and if the passenger is on their start floor, is not yet traveling, and has not yet arrived
                passenger.traveling = 1 #when a passenger is picked up they begin traveling;(0=not traveling, 1= traveling)
                self.load += 1 #increases the count of people that the elevator is carrying                
                self.time += 3 #this is the amount of time it takes to pick up one passenger        
           
              
    #used in the first strategy to unload passengers
    def unload(self, passengers):
        traveling_passengers = [passenger for passenger in passengers if passenger.traveling == 1] #traveling passengers
        for i, passenger in enumerate(traveling_passengers):
            if (self.position == passenger.end) & (passenger.traveling == 1): #these are the conditions when a passenger reaches their destination
                passenger.traveling = 0 #when the passenger gets off, they're no longer traveling
                passenger.arrived = 1 #when the passenger gets off, they have arrived
                self.load -= 1 #when a passenger gets off, the elevator load decreases by 1 person
                self.time += 3 #this is the amount of time that it takes to unload one passenger
  
    
    #used in the second stategy to pick up passengers. It only does one at a time, as a different method that incorporates it ("stop_check") already loops through all pasengers.
    #otherwise,this method is very similar to the "pick_up" method presented and explained above
    def pick_up_one(self, passenger): 
        if (self.position == passenger.start) & (passenger.traveling == 0) & (passenger.arrived == 0) & (self.load < self.max_capacity):
            passenger.traveling = 1
            self.load += 1
            self.time += 3

    #used in the second stategy to unload passengers. It only does one at a time, as a different method that incorporates it ("stop_check") already loops through all pasengers.
    #otherwise,this method is very similar to the "unload" method presented and explained above
    def unload_one(self, passenger):
        if (self.position == passenger.end) & (passenger.traveling == 1): 
            passenger.traveling = 0
            passenger.arrived = 1
            self.load -= 1
            self.time += 3 
        
    #used in the second strategy to check if any passengers want to load or unload at a given floor.
    #if at least one does, the elevator stops at that floor to unload and load, 
    #otherwise, the elevator continues to the next floor in the direction that it was going
    def stop_check(self,passengers):    
        for passenger in passengers:
            if ((self.position == passenger.start) & (passenger.traveling == 0) & (passenger.arrived == 0) & (self.load < self.max_capacity)) or ((self.position == passenger.end) & (passenger.traveling == 1)): #this checks if any passengers want to get on or off at the given floor
                if (self.door == 0):  
                    self.time += 12 #this is how long it takes to stop on a floor regardless of if any passengers load or unload
                    self.door = 1 #open door so passengers can enter
                self.unload_one(passenger) #unload passengers
                self.pick_up_one(passenger) #load passengers
        self.door = 0 #close door
    
    #used in the third strategy. 
    #it is used to identify the highest floor that any unarrived passengers are waiting on or wish to travel to.
    def passenger_max_floor(self, passengers):
        unarrived_passengers = [passenger for passenger in passengers if passenger.arrived == 0] #identifies all unarrived passengers
        sources = [passenger.start for passenger in unarrived_passengers] #indentifies everywhere that passengers are waiting
        destinations = [passenger.end for passenger in unarrived_passengers] #identifies everywhere that passengers wish to go
        return max(sources + destinations) #identifies the highest desired/needed floor
     
    #used in the third strategy. 
    #it is used to identify the lowest floor that any unarrived passengers are waiting on or wish to travel to.
    #this method works similarly to the one above ("passenger_max_floor"), but identifies the lowest floor instead of the highest.
    def passenger_min_floor(self, passengers):
        unarrived_passengers = [passenger for passenger in passengers if passenger.arrived == 0]
        sources = [passenger.start for passenger in unarrived_passengers] 
        destinations = [passenger.end for passenger in unarrived_passengers] 
        return min(sources + destinations)

#This is the Passenger class
class Passenger:
  
    #sets the initial values of the Passenger class
    def __init__(self,num_floors):
        self.start = randint(0,num_floors) #selects random start floor
        self.end = choice([i for i in range(0,num_floors) if i != self.start]) #selects random end floor that cannot be the same as the start floor       
        self.traveling = 0 #checks if the passenger is traveling; The passenger is not traveling at the beginning of the simulation (0 = not traveling, 1 = traveling)
        self.arrived = 0 #checks if the passenger has arrived; The passenger has not arrived at the beginning of the simulation (0 = not arrived, 1 = arrived)

#This is the Building class
class Building:
    """
    The building sets the stage for the simulation.
    The building has a number of specified floors.
    The building has a number of specified passengers and 1 elevator.
    """
    
    #sets the inital values of the Building class
    def __init__(self, num_floors = 10, num_passengers = 20, elevator_capacity = 10, elevator_position = 0):
        self.num_floors = num_floors #number of floors in the building
        self.passengers = [] #sets the passengers in the building
        for i in range(num_passengers):
            self.passengers.append(Passenger(num_floors)) 
        self.elevator = Elevator(elevator_capacity,elevator_position) #the default max capacity is 10 and the elevator starts on the lowest floor (0)
        
    #identifies when all passengers have arrived at their destination
    def check_end_simulation(self):
        unarrived_passengers = [passenger for passenger in self.passengers if passenger.arrived == 0] #If unarrived_passengers is an empty list, then simulation ends
        if unarrived_passengers:
            return False
        else:
            arrived = [passenger.arrived for passenger in self.passengers] #vizualizes the arrival status (0= not arrived, 1=arrived) of all passengers
            return True
        

In [0]:
## FIRST STRATEGY ##

class DefaultBuilding(Building):
    def run(self):
        """ Run the simulation
        Elevator starts at the ground floor
        Move up one floor: stop, unload, load (unload and load only if there are passengers to do so)
        Continue moving up, stopping, unloading, and loading on every floor regardless of where the passengers want to go
        When it reaches the top of the building, begin going down one floor at a time in the same manner
        Continue this process as many times as needed until all passengers are delivered
        Note: There cannot be more than 10 people in the elevator at one time. Passengers will not load if elevator is at max capacity.
        """
        elevator = self.elevator
        passengers = self.passengers
        random.seed(9) #sets seed for random, allows for comparison between strategies
        
        arrived = [] #arrival status of all the passengers
        arrived = [passenger.arrived for passenger in passengers] #the initial arrival status is 0 = not arrived for all passengers
        print("the arrival status of all passengers is ", arrived)

        check_arrived = False #Indicates whether or not all passengers have arrived at their destinations.
        iteration = 1 #one iteration involves going all the way to the top of the building and then all the way back down. It starts at 1.
        
        while check_arrived == False:
            print("iteration number:", iteration)
            for floor in range(self.num_floors): #the process is repeated for each floor from the ground to the top
                #go up 
                self.elevator.move_one(1) #goes up one floor
                self.elevator.unload(passengers) #unloads passengers
                self.elevator.pick_up(passengers) #picks up passengers
                self.elevator.time += 12 #time it takes to stop on a floor/for the doors to open and close

            for floor in (range(self.num_floors)): #the process is repeated for each floor from the top to the ground
                #go down
                self.elevator.move_one(-1) #goes down one floor
                self.elevator.unload(passengers) #unloads passengers
                self.elevator.pick_up(passengers) #loads passengers
                self.elevator.time += 12 #time it takes to stop on a floor/for the doors to open and close

            arrived = [passenger.arrived for passenger in passengers] #vizualizes the arrival status (0= not arrived, 1=arrived) of all passengers
            print("the arrival status of all passengers is ", arrived)

            check_arrived = self.check_end_simulation() #checks if all passengers have arrived

            iteration +=1 #an iteration involves going all the way to the top of the building and all the way back down

        print ("All passengers arrived at their destination, the total time it takes is %d seconds" %(elevator.time)) #prints when all passengers have arrived at their destiations

        


In [0]:
## SECOND STRATEGY ##
""" Run the simulation
    Starts at the ground floor
    Moves up one floor: checks if any passengers want to load or unload on that floor
    If they don't, the elevator doesn't stop and continues to the next floor up
    If they do, it stops, unloads, then loads.
    The elevator continues this process until it reaches the top of the building, when it turns around and continues the process going down one floor at a time
    It continues this process as many times as needed until all passengers have arrived at their destinations
    Note: There cannot be more than 10 people in the elevator at one time. Passengers will not load if elevator is at max capacity.
    The main difference betweeen strategies 1 and 2 is that in the second strategy the elevator will *not* stop if no passengers want to load or unload on the given floor,
    whereas it always stops on every floor in the first strategy
    """

class FirstBuilding(Building):
    def run(self):
        elevator = self.elevator
        passengers = self.passengers
        random.seed(9)

        arrived = [] #arrival status of all the passengers
        arrived = [passenger.arrived for passenger in passengers] #the initial arrival status is 0 = not arrived for all passengers
        print("the arrival status of all passengers is ", arrived)

        check_arrived = False #indicates whether or not all passengers have arrived at their destinations.
        iteration = 1 #one iteration involves going all the way to the top of the building and then all the way back down. It starts at 1.
        
        while check_arrived == False:
            print("iteration number:", iteration)

            #go up
            for floor in range(self.num_floors): #the process is repeated for each floor from the ground to the top
                elevator.move_one(1) #moves up one floor
                elevator.stop_check(passengers) #checks to see if anyone wants to load or unload. If so, they do, otherwise the elevator goes to the next floor

            #go down
            for floor in range(self.num_floors): #the process is repeated for each floor from the top to the ground
                elevator.move_one(-1) #moves down one floor
                elevator.stop_check(passengers) #checks to see if anyone wants to load or unload. If so, they do, otherwise the elevator goes to the next floor

            arrived = [passenger.arrived for passenger in passengers] #vizualizes the arrival status (0= not arrived, 1=arrived) of all passengers
            print("the arrival status of all passengers is ", arrived)

            check_arrived = self.check_end_simulation() #checks to see if all passengers have arrived

            iteration +=1

        print ("All passengers arrived at their destination, the total time it takes is %d seconds" %(elevator.time)) #prints when all passengers have arrived to their intended destinations



In [0]:
## THIRD STRATEGY ##
""" Run the simulation
    Similar to the second strategy (see above). 
    The main difference between strategies 2 and 3 is that in the third strategy the elevator will not continue in its current direction (up or down) if there is no one to pick up or drop off in that direction.
    In the second strategy the elevator always goes all the way up and then all the way back down.
    """

class SecondBuilding(Building):
    def run_elevator(self, min_floor, max_floor, direction):
        for floor in range(min_floor, max_floor): #go up or down, only go as far as the distance between the highest floor and lowest floor that need to be accessed by passengers
            self.elevator.move_one(direction) #move up one floor
            self.elevator.stop_check(self.passengers) #checks to see if anyone wants to load or unload. If so, they do, otherwise the elevator goes to the next floor
            check_arrived = self.check_end_simulation() #after moving every floor
            if check_arrived == True: #check if everyone has arrived at their destination
                return True 
        return False #if not then return False
    def run(self):
        elevator = self.elevator
        passengers = self.passengers
        random.seed(9)

        arrived = [] #arrival status of all the passengers        
        check_arrived = False #Indicates whether or not all passengers have arrived at their destinations.
        iteration = 1

        while check_arrived == False:
            print("iteration number:", iteration)
            
            arrived = [passenger.arrived for passenger in passengers] #vizualizes the arrival status (0= not arrived, 1=arrived) of all passengers
            print("the arrival status of all passengers is ", arrived)
            
            max_floor = elevator.passenger_max_floor(passengers) #highest floor in the building with passengers on it or at which passengers want to be delivered
            print "Highest floor with waiting passenger is", max_floor
            
            min_floor = elevator.passenger_min_floor(passengers) #lowest floor in the building with passengers on it or at which passengers want to be delivered
            print "Lowest floor with waiting passenger is", min_floor
            
            check_arrived = self.run_elevator(min_floor, max_floor, 1) #ends simulation if all passengers have arrived. 
            if check_arrived == True:
                arrived = [passenger.arrived for passenger in passengers] #vizualizes the arrival status (0= not arrived, 1=arrived) of all passengers
                print("the arrival status of all passengers is ", arrived)
                break
           
            check_arrived = self.run_elevator(min_floor, max_floor, -1)
            if check_arrived == True:
                arrived = [passenger.arrived for passenger in passengers] #vizualizes the arrival status (0= not arrived, 1=arrived) of all passengers
                print("the arrival status of all passengers is ", arrived)
                break  

            iteration +=1

        print ("All passengers arrived at their destination, the total time it takes is %d seconds" %(elevator.time)) #prints when all passengers have arrived to their intended destinations
        



In [16]:
#Running the simulation:
#Default buildings have 10 floors and 20 passengers
#Default elevator has a capacity of 10 and starts on floor 0 (groundfloor)

if __name__ == "__main__":
    #runs elevator with first strategey
    print "First Strategy"
    df = DefaultBuilding()
    df.run()
    
    
    #runs elevator with second strategey
    print "\n\nSecond Strategy"
    df = FirstBuilding()
    df.run()
    
    #runs elevator with third strategey
    print "\n\nThird Strategy"
    df = SecondBuilding()
    df.run()

First Strategy
('the arrival status of all passengers is ', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
('iteration number:', 1)
('the arrival status of all passengers is ', [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])
('iteration number:', 2)
('the arrival status of all passengers is ', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
All passengers arrived at their destination, the total time it takes is 720 seconds


Second Strategy
('the arrival status of all passengers is ', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
('iteration number:', 1)
('the arrival status of all passengers is ', [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])
('iteration number:', 2)
('the arrival status of all passengers is ', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
All passengers arrived at their destination, the total time it takes is 480 seconds


Third Strategy
('iteration number:', 1)
('the arrival s